In [1]:
#Needed in general
# from __future__ import division

import numpy as np
# import matplotlib.pyplot as plt
import sys
import h5py as h5
# import gc

#Quick fudge to make import from ../Scripts work
sys.path.append('../../Scripts')

#Custom imports
import ClassCOMPAS     as CC ###
# import ClassFormationChannels as FC 


# import script that has formation channel classification functions:
from PostProcessingScripts import * 
# from ClassFormationChannels_5mainchannels import * 



# from matplotlib.ticker import AutoMinorLocator, MultipleLocator
# minor = AutoMinorLocator()
# import math # for tan function and degrees function

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


In [2]:
import string

# some label lists for different models 
nModels=15
BPSnameslist = list(string.ascii_uppercase)[0:nModels]
# I put every data file in an own directory, with the following names: 
modelDirList = ['fiducial', 'massTransferEfficiencyFixed_0_25', 'massTransferEfficiencyFixed_0_5', 'massTransferEfficiencyFixed_0_75', \
               'unstableCaseBB', 'alpha0_5', 'alpha2_0', 'fiducial', 'rapid', 'maxNSmass2_0', 'maxNSmass3_0', 'noPISN',  'ccSNkick_100km_s', 'ccSNkick_30km_s', 'noBHkick' ]

alphabetDirDict =  {BPSnameslist[i]: modelDirList[i] for i in range(len(BPSnameslist))}


physicalNamesBPSmodels = [r'\textbf{fiducial}',\
                           r'$\beta=0.25$', r'$\beta=0.5$',  r'$\beta=0.75$',r'\textbf{unstable case BB}',\
                           r'$\alpha_{\rm{CE}}=0.5$',  r'$\alpha_{\rm{CE}}=2$', r'\textbf{optimistic CE}',\
                          r'\textbf{rapid SN}', r'$\rm{max} \ m_{\rm{NS}}=2.0\,\rm{M}_{\odot}$', r'$\rm{max} \ m_{\rm{NS}}=3.0\,\rm{M}_{\odot}$',\
                          r'\textbf{no PISN}', r'\textbf{SN} '+ r'$\sigma_{\rm{cc}}=100\,\rm{km}\,\rm{s}^{-1}$',r'\textbf{SN} '+ r'$\sigma_{\rm{cc}}=30\,\rm{km}\,\rm{s}^{-1}$',\
                          r'\textbf{SN} '+ r'$v_{\rm{k,BH}}=0\,\rm{km}\,\rm{s}^{-1}$' ]



alphabetPhysicalNameDict =  {BPSnameslist[i]: physicalNamesBPSmodels[i] for i in range(len(BPSnameslist))}

## Calculating statistical uncertainty on rate estimates 

The code below calculates the statistical (Poisson) uncerttainty on the rate estimates from each of our simulations. In the end it quotes also the highest percentage of the uncertainty (as a percentage of total BHNS rate)

In [4]:
mainpath = '/Volumes/Andromeda/DATA/AllDCO_bugfix/'

# I want figure for BHNS binaries
DCOtype='BHNS'

max_uncertainty = 0

for ind_bps, bps_model in enumerate(BPSnameslist):
    
    
    print()
    print('-----------------')
    print('now doing BPS model %s, %s'%(bps_model, alphabetDirDict[bps_model]))



    
    
    # get fdata from path to this particularly bps file based on how I named the different models
    path = mainpath + str(alphabetDirDict[bps_model]) + '/COMPASCompactOutput_' + DCOtype +'_' + bps_model +'.h5'
    fdata = h5.File(path)


    OPTIMISTIC=False
    if bps_model=='H':
        # only for model H we want optimistic CE selection
        OPTIMISTIC=True
        print('doing optimistic version of fiducial')    

    
    




    ##### Setting DATA file #####
    #But I want only within Hubble time 
    Data            = CC.COMPASData(path=path, lazyData=True, Mlower=5., \
                     Mupper=150., binaryFraction=1)
    Data.setCOMPASDCOmask(types=DCOtype,  withinHubbleTime=True, optimistic=OPTIMISTIC)
    Data.setCOMPASData()
    SeedsHubble    = Data.seeds[Data.Hubble==True]




    # calculate the uncertainty on the rate from poisson scatter: 
    print('total weight BHNS mergers in COMPAS simulation =', np.sum(Data.weight))
    rate = np.sum(Data.weight)
    Nzeros = int(53*1E6 - len(Data.weight))
    all_weights = np.concatenate((np.zeros(Nzeros), Data.weight))
    var_rate = np.std(all_weights, ddof=1) * np.sqrt(1E6)

    print('fractional uncertainty on total weight BH-NS =', var_rate/rate)
    print('percentage uncertainty on total weight BH-NS =', 100*var_rate/rate, '%')

    print('STROOPWAFEL gain =', len((Data.weight))/np.sum(Data.weight))
    
    
    max_uncertainty = max(max_uncertainty, 100*var_rate/rate)
    
    
print()
print('the overall max statistical uncertainty is ', max_uncertainty, '%')




-----------------
now doing BPS model A, fiducial
weighted samples :-D
Remember to  self.setCOMPASDCOmask() and self.setCOMPASData()
total weight BHNS mergers in COMPAS simulation = 23174.234060106566
fractional uncertainty on total weight BH-NS = 0.00025328472928200175
percentage uncertainty on total weight BH-NS = 0.025328472928200178 %
STROOPWAFEL gain = 65.82970535480061

-----------------
now doing BPS model B, massTransferEfficiencyFixed_0_25
weighted samples :-D
Remember to  self.setCOMPASDCOmask() and self.setCOMPASData()
total weight BHNS mergers in COMPAS simulation = 15581.352363435695
fractional uncertainty on total weight BH-NS = 0.00035310477755190695
percentage uncertainty on total weight BH-NS = 0.035310477755190695 %
STROOPWAFEL gain = 47.39877404564081

-----------------
now doing BPS model C, massTransferEfficiencyFixed_0_5
weighted samples :-D
Remember to  self.setCOMPASDCOmask() and self.setCOMPASData()
total weight BHNS mergers in COMPAS simulation = 8435.3299939